In [1]:
# Add the parent directory to Python path so we can import from core
import sys
import os
# Get the current working directory and go up one level to the backend directory
backend_dir = os.path.dirname(os.getcwd())
if backend_dir not in sys.path:
    sys.path.append(backend_dir)

In [2]:
from core.issue import IssueContext
from core.agents.communications import CommunicationsAgent
from core.llm import LLMClient

# Create new agent instance to test
comms_agent = CommunicationsAgent(llm_client=LLMClient(), issue_id="123")

# Dummy Data
test = {
        'id': 'test_4f1452dc',
        'name': 'Fuel Level Check',
        'description': 'Ask the user to check the fuel gauge or inspect the fuel tank to confirm whether the truck has enough fuel. If the tank is empty, refill it; if it has fuel, proceed to the next step.',
        'rationale': 'A lack of fuel is a common reason the engine cranks but does not start. Verifying fuel level is quick and inexpensive, and if fuel is present, it narrows the diagnosis to electrical or mechanical issues.',
        'outcomes': {
            'type_of_outcome': 'string', 
            'outcome_data': "possible values: 'empty', 'not empty', 'unknown'"
        }, 
        'result': 'not empty'
    }

to_send = await comms_agent.communicate_test(test)

print(to_send, flush=True)

We need to produce JSON with test_text, instructions, result field label etc. The test description: Ask user to check fuel gauge or inspect tank to confirm whether truck has enough fuel. If empty, refill. If has fuel proceed. The outcome field: possible values 'empty', 'not empty', 'unknown'. The result already given "not empty". Provide test text: "Please check your truck’s fuel level to confirm whether it has enough fuel." Instructions: maybe one step: "Check the fuel gauge or open the tank to see if fuel is present." Provide result field label: maybe "Fuel Level". Type: array with options. test_instructions array empty? Since it's a single question? Actually instructions step: "1. Check fuel gauge or tank. If empty, refill. If not empty, proceed." We can give one step. test_result_field_label "Fuel Level". test_result_field_type "array". Options ['empty', 'not empty', 'unknown'].

Safety_and_warnings: maybe "Do not operate the vehicle until fuel is confirmed." Also "If the fuel gaug

In [2]:
from core.issue import IssueContext
from core.agents.diagnostics import DiagnosticsAgent, LLMDiagnosticsAgent
from core.llm import LLMClient

# Create new agent instance to test
diagnostics_agent = LLMDiagnosticsAgent(llm_client=LLMClient())

# Dummy Data
log = [
    {
        "id": "test_1",
        "name": "Issue Description",
        "description": "This is the issue description initially given by the user.",
        "rationale": "",
        "outcomes": {},
        "result": "My truck revs up when I put the keys in the ignition but it won't start."
    },
    {
        'id': 'test_4f1452dc',
        'name': 'Fuel Level Check',
        'description': 'Ask the user to check the fuel gauge or inspect the fuel tank to confirm whether the truck has enough fuel. If the tank is empty, refill it; if it has fuel, proceed to the next step.',
        'rationale': 'A lack of fuel is a common reason the engine cranks but does not start. Verifying fuel level is quick and inexpensive, and if fuel is present, it narrows the diagnosis to electrical or mechanical issues.',
        'outcomes': {
            'type_of_outcome': 'string', 
            'outcome_data': "possible values: 'empty', 'not empty', 'unknown'"
        }, 
        'result': 'not empty'
    }
]

probabilities = [
    {'diagnosis': 'Starter Motor Failure', 'probability': 2.0}, 
    {'diagnosis': 'Fuel Delivery Problem', 'probability': 2.0}, 
    {'diagnosis': 'Ignition System Failure', 'probability': 1.2}, 
    {'diagnosis': 'Low Battery', 'probability': 0.5}, 
    {'diagnosis': 'Engine Seizure', 'probability': 0.5}
]





new_probabilities, next_test = await diagnostics_agent.run(
    diagnosis_probabilities=probabilities,
    tests_log=log
)

print(new_probabilities)
print(next_test)


We have prior hypotheses and probabilities. We received most recent test: Fuel Level Check, result "not empty". We must update probabilities multiplicatively.

We have rules: strong support ×2.0; some support ×1.2; neutral ×1.0; some contradiction ×0.8; strong contradiction ×0.5.

We need to determine how the result 'not empty' affects each hypothesis. Let's analyze each hypothesis:

1. Starter Motor Failure: If starter motor fails, engine cranks but doesn't start. Fuel level irrelevant. Fuel level not empty indicates that fuel isn't the problem. So result does NOT support Starter Motor Failure; it doesn't contradict it either? Actually, a not-empty fuel means engine may crank but still not start due to other issue. The result does not influence starter motor failure: it's neutral. So factor ×1.0.

2. Fuel Delivery Problem: This is contradicted by not empty fuel? Fuel delivery problem might be fuel pump or filter, but fuel presence doesn't rule out delivery issues; but if fuel tank not